In [78]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from tpot import TPOTClassifier
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import signal 
import pickle


import sklearn.metrics
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [79]:
dataframe_hrv = pd.read_csv("dataframe_hrv.csv")
dataframe_hrv = dataframe_hrv.reset_index(drop = True)

In [80]:
dataframe_hrv.head()

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,stress,time,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
0,-0.001974,-0.004737,77.815789,10.801842,12.529684,2.417132,10.889447,0.614632,NaN,12.529684,0.0,12.529684,0.973684,0.617297,3.558630e-02,0.015203,0.055556,0.001238,0.0,0.000696,0.000407,0.000135,3.00200
1,0.002935,-0.004457,101.978261,10.750609,30.503500,2.417109,11.251065,0.647826,NaN,30.503500,0.0,30.503500,0.978261,0.647889,1.354660e-02,0.013858,0.045455,0.000144,0.0,0.000009,0.000060,0.000075,0.79371
2,0.006745,-0.003426,104.957447,10.557234,52.523021,2.226872,11.379638,0.646383,NaN,52.523021,0.0,52.523021,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
3,-0.004043,-0.002532,87.702128,10.640128,74.402170,2.173021,11.470830,0.645000,NaN,74.402170,0.0,74.402170,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
4,0.012745,-0.004426,88.829787,10.699319,96.219617,2.017106,11.135255,0.645000,NaN,96.219617,0.0,96.219617,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN


In [81]:
dataframe_hrv['stress'] = np.where(dataframe_hrv['stress']>=0.5,1,0)


In [82]:

#dataframe_hrv = dataframe_hrv.reset_index()
dataframe_hrv = dataframe_hrv.replace([np.inf, -np.inf], np.nan)
dataframe_hrv[~np.isfinite(dataframe_hrv)] = np.nan
#     df.plot( y=["HR"])


In [83]:
dataframe_hrv.head()

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,stress,time,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
0,-0.001974,-0.004737,77.815789,10.801842,12.529684,2.417132,10.889447,0.614632,NaN,12.529684,0,12.529684,0.973684,0.617297,3.558630e-02,0.015203,0.055556,0.001238,0.0,0.000696,0.000407,0.000135,3.00200
1,0.002935,-0.004457,101.978261,10.750609,30.503500,2.417109,11.251065,0.647826,NaN,30.503500,0,30.503500,0.978261,0.647889,1.354660e-02,0.013858,0.045455,0.000144,0.0,0.000009,0.000060,0.000075,0.79371
2,0.006745,-0.003426,104.957447,10.557234,52.523021,2.226872,11.379638,0.646383,NaN,52.523021,0,52.523021,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
3,-0.004043,-0.002532,87.702128,10.640128,74.402170,2.173021,11.470830,0.645000,NaN,74.402170,0,74.402170,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
4,0.012745,-0.004426,88.829787,10.699319,96.219617,2.017106,11.135255,0.645000,NaN,96.219617,0,96.219617,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN


In [84]:
dataframe_hrv['HR'].fillna((dataframe_hrv['HR'].mean()), inplace=True)
dataframe_hrv['HR'] = signal.medfilt(dataframe_hrv['HR'],13) 


In [85]:
dataframe_hrv.head()

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,stress,time,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
0,-0.001974,-0.004737,77.815789,10.801842,12.529684,2.417132,10.889447,0.614632,NaN,12.529684,0,12.529684,0.973684,0.617297,3.558630e-02,0.015203,0.055556,0.001238,0.0,0.000696,0.000407,0.000135,3.00200
1,0.002935,-0.004457,79.521739,10.750609,30.503500,2.417109,11.251065,0.647826,NaN,30.503500,0,30.503500,0.978261,0.647889,1.354660e-02,0.013858,0.045455,0.000144,0.0,0.000009,0.000060,0.000075,0.79371
2,0.006745,-0.003426,87.319149,10.557234,52.523021,2.226872,11.379638,0.646383,NaN,52.523021,0,52.523021,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
3,-0.004043,-0.002532,87.702128,10.640128,74.402170,2.173021,11.470830,0.645000,NaN,74.402170,0,74.402170,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
4,0.012745,-0.004426,87.702128,10.699319,96.219617,2.017106,11.135255,0.645000,NaN,96.219617,0,96.219617,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN


In [76]:
#df.plot( y=["HR"])
dataframe_hrv=dataframe_hrv.fillna(dataframe_hrv.mean())
#dataframe_hrv = missing_values(dataframe_hrv)
#

AttributeError: 'NoneType' object has no attribute 'fillna'

In [86]:
dataframe_hrv.head()

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,stress,time,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
0,-0.001974,-0.004737,77.815789,10.801842,12.529684,2.417132,10.889447,0.614632,NaN,12.529684,0,12.529684,0.973684,0.617297,3.558630e-02,0.015203,0.055556,0.001238,0.0,0.000696,0.000407,0.000135,3.00200
1,0.002935,-0.004457,79.521739,10.750609,30.503500,2.417109,11.251065,0.647826,NaN,30.503500,0,30.503500,0.978261,0.647889,1.354660e-02,0.013858,0.045455,0.000144,0.0,0.000009,0.000060,0.000075,0.79371
2,0.006745,-0.003426,87.319149,10.557234,52.523021,2.226872,11.379638,0.646383,NaN,52.523021,0,52.523021,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
3,-0.004043,-0.002532,87.702128,10.640128,74.402170,2.173021,11.470830,0.645000,NaN,74.402170,0,74.402170,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
4,0.012745,-0.004426,87.702128,10.699319,96.219617,2.017106,11.135255,0.645000,NaN,96.219617,0,96.219617,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN


In [87]:
selected_x_columns = ['HR','interval in seconds','AVNN', 'RMSSD', 'pNN50', 'TP', 'ULF', 'VLF', 'LF', 'HF','LF_HF']

X = dataframe_hrv[selected_x_columns]
y = dataframe_hrv['stress']

display(X.columns)
display(X.describe())
display(X.shape)

Index(['HR', 'interval in seconds', 'AVNN', 'RMSSD', 'pNN50', 'TP', 'ULF',
       'VLF', 'LF', 'HF', 'LF_HF'],
      dtype='object')

,HR,interval in seconds,AVNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
count,4129.000000,4129.000000,4129.000000,4129.000000,4129.000000,3806.000000,3933.000000,4002.000000,4002.000000,4002.000000,58.000000
mean,81.144252,0.788440,0.788020,0.027223,0.034246,0.060604,0.057902,0.000728,0.000003,0.000001,3.555695
std,10.871620,0.109768,0.109363,0.016964,0.025153,0.059267,0.059179,0.003244,0.000031,0.000010,2.080121
min,60.558824,0.527944,0.528170,0.000000,0.000000,0.000008,0.000000,0.000000,0.000000,0.000000,0.418526
25%,73.394737,0.718930,0.720548,0.013743,0.025641,0.014438,0.009625,0.000000,0.000000,0.000000,2.220580
50%,78.973684,0.789732,0.790125,0.022889,0.027027,0.032922,0.027761,0.000000,0.000000,0.000000,3.169260
75%,87.527778,0.869053,0.868135,0.043000,0.030303,0.105434,0.104521,0.000000,0.000000,0.000000,4.865465
max,115.446809,1.040088,1.038360,0.080981,0.257143,0.329891,0.329891,0.035841,0.000616,0.000261,9.117240


(4129, 11)

In [88]:
def do_tpot(generations=5, population_size=10,X='',y=''):

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=0.80, test_size=0.20)

    tpot = TPOTClassifier(generations=generations, population_size=population_size, verbosity=2,cv=3)
    tpot.fit(X_train, y_train)
    print(tpot.score(X_test, y_test))
    tpot.export('tpot_pipeline.py')
    return tpot

tpot_classifer = do_tpot(generations=10, population_size=20,X=X,y=y)

Imputing missing values in feature set


Generation 1 - Current best internal CV score: 0.7465988147645971
Generation 2 - Current best internal CV score: 0.7465988147645971
Generation 3 - Current best internal CV score: 0.7472073494605231
Generation 4 - Current best internal CV score: 0.7502340769739159
Generation 5 - Current best internal CV score: 0.7502340769739159
Generation 6 - Current best internal CV score: 0.7526547398851409
Generation 7 - Current best internal CV score: 0.7568916649845553
Generation 8 - Current best internal CV score: 0.7626426416706167
Generation 9 - Current best internal CV score: 0.7626426416706167
Generation 10 - Current best internal CV score: 0.7626426416706167

Best pipeline: KNeighborsClassifier(DecisionTreeClassifier(FeatureAgglomeration(input_matrix, affinity=euclidean, linkage=complete), criterion=gini, max_depth=4, min_samples_leaf=20, min_samples_split=4), n_neighbors=99, p=1, weights=distance)
Imputing missing values in feature set
0.7966101694915254


In [101]:
def plotFitBitReading(dfnewHRV='', predictor = "none",selected_x_columns=''):
    dfnewHRV = missing_values(dfnewHRV)
    dfnewPol = dfnewHRV[selected_x_columns].fillna(0)

    print(dfnewPol.columns)
    print(dfnewPol.shape)
    pred = predictor.predict_proba(dfnewPol)
    
    dfpred = pd.DataFrame(pred)

    dfpred.columns = [["FALSE","TRUE"]]
    dfpred['stress'] = np.where(dfpred["TRUE"] > 0.5, 1, np.nan)

    
    dfnewHRV["stress"] = dfpred["stress"]
    dfnewHRV.loc[dfnewHRV["steps"] > 0, 'stress'] = np.nan
    #mark is to mark the RR peaks as stress
    dfnewHRV.loc[dfnewHRV["stress"] == 1, 'stress'] = dfnewHRV['interval in seconds'] 
    dfnewHRV.loc[dfnewHRV["steps"] > 0, 'moving'] = dfnewHRV['interval in seconds'] 
    dfnewHRV["minutes"] = (dfnewHRV['newtime']/60)/1000
    
    from itertools import cycle, islice
    my_colors = list(islice(cycle(['b', 'r', 'y', 'k']), None, len(dfnewHRV)))
    plot = dfnewHRV.plot(x="minutes", y=['interval in seconds',"stress", "moving"],color=my_colors)
    
    fig = plot.get_figure()

    

In [102]:
import glob

for filename in glob.iglob('dataset/**/*.csv', recursive=True):
    if 'dfnew' in filename:
        print(filename)

dataset\American Horror Story\female_24_years_american_horror_story\dfnewHRV.csv
dataset\American Horror Story\male_22years_american_horror_story\dfnewHRVmale_22yearsAmericanHorrorStory.csv
dataset\American Horror Story\Male_25_years_american_horror_story\dfnewHRV_Male_25_years_american_horror_story.csv
dataset\Vikings\Female_24_years_old\dfnewHRV.csv
dataset\Vikings\Female_33_years_old\dfnewHRV_Female_33_years_old_vikings.csv
dataset\Vikings\Male_21_years\dfnewHRV.csv
dataset\Vikings\male_22years\dfnewHRVVIKINGSFULLSERIES.csv
dataset\Vikings\Male_23_years_old\dfnewHRVMale_23_years_old.csv


In [103]:
input_df = pd.read_csv('dataset/Vikings/Female_24_years_old/dfnewHRV.csv')
input_df

,calories,distance,floors,heart,interval in seconds,newtime,nntime,rrmedfilt,steps,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF,HR
0,1.504656,0.000000,0.000000,84.600000,0.688867,9.563733e+03,9.563733e+03,687.033333,0.000000,0.966667,0.690552,0.071482,0.033190,0.142857,0.004869,0.004869,0.0,0.0,0.0,NaN,84.600000
1,1.399680,0.000000,0.000000,84.000000,0.707172,3.056293e+04,3.056293e+04,705.758621,0.000000,0.965517,0.708536,0.022942,0.022299,0.037037,0.000490,0.000490,0.0,0.0,0.0,NaN,84.000000
2,1.399680,0.000000,0.000000,84.000000,0.708759,5.118469e+04,5.118469e+04,707.034483,0.000000,0.965517,0.710179,0.039112,0.029066,0.037037,0.001425,0.001425,0.0,0.0,0.0,NaN,84.000000
3,1.399680,0.000000,0.000000,79.961538,0.783423,7.155296e+04,7.155296e+04,782.692308,0.000000,0.961538,0.786160,0.080011,0.043496,0.291667,0.005829,0.005829,0.0,0.0,0.0,NaN,79.961538
4,1.399680,0.000000,0.000000,79.000000,0.820000,9.224544e+04,9.224544e+04,810.920000,0.000000,0.960000,0.820542,0.052691,0.048884,0.434783,0.002628,0.002628,0.0,0.0,0.0,NaN,79.000000
5,1.399680,0.000000,0.000000,79.000000,0.774654,1.125171e+05,1.125171e+05,772.423077,0.000000,0.961538,0.777080,0.064969,0.046112,0.166667,0.003879,0.003879,0.0,0.0,0.0,NaN,79.000000
6,1.399680,0.000000,0.000000,80.777778,0.753926,1.327373e+05,1.327373e+05,754.148148,0.000000,0.962963,0.753731,0.027161,0.026085,0.040000,0.000697,0.000697,0.0,0.0,0.0,NaN,80.777778
7,1.399680,0.000000,0.000000,81.000000,0.735571,1.531042e+05,1.531042e+05,736.750000,0.000000,0.964286,0.735630,0.015428,0.021868,0.000000,0.000225,0.000225,0.0,0.0,0.0,NaN,81.000000
8,1.399680,0.000000,0.000000,81.000000,0.736821,1.735051e+05,1.735051e+05,734.535714,0.000000,0.964286,0.737074,0.050469,0.027346,0.076923,0.002415,0.002415,0.0,0.0,0.0,NaN,81.000000
9,1.399680,0.000000,0.000000,75.240000,0.818920,1.944716e+05,1.944716e+05,819.680000,0.000000,0.960000,0.818792,0.042744,0.035034,0.173913,0.001715,0.001715,0.0,0.0,0.0,NaN,75.240000


In [107]:
plotFitBitReading(input_df,tpot_classifer,selected_x_columns)

TypeError: 'NoneType' object is not subscriptable

In [113]:
predictor = tpot_classifer
new_in = input_df[selected_x_columns]
pred = predictor.predict_proba(new_in)

Imputing missing values in feature set


In [114]:
pred

array([[0.87735199, 0.12264801],
       [0.88832418, 0.11167582],
       [0.8909304 , 0.1090696 ],
       [0.41910478, 0.58089522],
       [0.36301677, 0.63698323],
       [0.3977619 , 0.6022381 ],
       [0.25578818, 0.74421182],
       [0.2843997 , 0.7156003 ],
       [0.77731941, 0.22268059],
       [0.27663693, 0.72336307],
       [0.65481577, 0.34518423],
       [0.56471914, 0.43528086],
       [0.52074388, 0.47925612],
       [0.52757676, 0.47242324],
       [0.39504522, 0.60495478],
       [0.33893771, 0.66106229],
       [0.33906413, 0.66093587],
       [0.40114798, 0.59885202],
       [0.32834259, 0.67165741],
       [0.5161763 , 0.4838237 ],
       [0.36387011, 0.63612989],
       [0.20612582, 0.79387418],
       [0.20615922, 0.79384078],
       [0.20629659, 0.79370341],
       [0.00908265, 0.99091735],
       [0.03483852, 0.96516148],
       [0.40428621, 0.59571379],
       [0.3423562 , 0.6576438 ],
       [0.33831573, 0.66168427],
       [0.19792946, 0.80207054],
       [0.

In [150]:

dfpred1 = pd.DataFrame(pred)



In [153]:

dfpred1 = np.where(dfpred1[1] < 0.33, "Low", (np.where(dfpred1[1] < 0.66, 'Medium', 'High')))
#dfpred1[1]
    

In [125]:
for i in dfpred["TRUE"]:
    if i < 0.33:
        i = 1
    elif i > 0.66:
        i = 3
    else:
        i = 2


TypeError: only integer scalar arrays can be converted to a scalar index

In [154]:
dfpred1

array(['Low', 'Low', 'Low', 'Medium', 'Medium', 'Medium', 'High', 'High',
       'Low', 'High', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium',
       'High', 'High', 'Medium', 'High', 'Medium', 'Medium', 'High',
       'High', 'High', 'High', 'High', 'Medium', 'Medium', 'High', 'High',
       'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High',
       'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High',
       'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High',
       'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High',
       'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High',
       'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High',
       'High', 'High', 'Low', 'High', 'Medium', 'Low', 'Low', 'High',
       'High', 'High', 'Medium', 'Medium', 'Medium', 'Medium', 'High',
       'High', 'High', 'High', 'High', 'Medium', 'Medium', 'Medium',
       'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium',
       